In [1]:
from IPython.display import display, Markdown
def latexify(x):
    out = '$' + x + '$'
    return out

def lprint(x):
    display(Markdown(latexify(latex(x))))

# Implicit Function Theorem

The implicit function theorem is a core ingredient in bifurcation theory, in this notebook we will explore how to apply it algorithmically 

## Statement of theorem

We will consider functions of the form:

$$f : \mathbb{R}^n+m \rightarrow \mathbb{R}^m $$

usually k-times continuously differentiable for $k \in \mathbb{N}$

The implicit function theorem is used to characterise the zero set of the function around a know solution, suppose (without loss of generality) that:

$$f(0) = 0$$

We then split $\mathbb{R}^n$ into the product of two spaces:

$$\mathbb{R}^{n+m} = \mathbb{R}^n \times \mathbb{R}^m$$ 

From here on we denote $\mathbb{R}^n = X$ and $\mathbb{R}^m = Y$

So that:

$$f : X \times Y \rightarrow \mathbb{R}^m$$

If we then suppose that the derivtative with respect to $Y$ is an isomorphism at 0, i.e:

$$\partial_{Y}f(0,0)$$ is invertible

Then the Implict Function Theorem states that there exists, open subsets of $X$ and $Y$ and a k-times differentiable function:

$$h : U_{X} \rightarrow U_{y}$$

That paramatrises the zero set of $f$ close to zero:

$$\{ (x,y) \in U_{X} \times U_{Y} \;| \;\; f(x,y) = 0 \} = \{ (x,h(x)) \;\; | \;\; x \in U_{X} \}$$

## Computing $h$

We will now construct an algorithm, that can produce a Taylor polynomial approximation of the $h$ function, assuming we know that $\mathbb{R}^{n+m}$ meets the conditions required.

Recall that:

$$h(x) = h(a) + \frac{h'(a)}{1!}(x-a) + \frac{h(a)}{2!}(x-a)^2 + \frac{h'(a)}{3!}(x-a)^3+\dotsb = \sum_{k=0}^\infty \frac{h^{\left(k\right)}(a)}{k!} (x-a)^k$$

is the k-th order Taylor expansion of $h$ at 0

We require a way to find the derivatives of $h$, specifically:

$\partial_{X}h$,     $\partial_{XX}h$,     $\partial_{XXX}h$ 

and so on

To do this observe that since $f(x, h(x)) = 0 \;\;\forall x \in U_{X}$

$$ \partial_{X}^{k}f(x,h(x)) = 0 \;\; \forall k \in \mathbb{N}, \forall x \in U_{X}$$

The first couple of applications of this idea are:

$$ 0 = f(x, h(x))$$

$$ 0 = f_{X} + f_{Y}h_{X}$$ 

$$ 0 = f_{XX} + 2f_{XY}h_{X} + f_{YY}h_{X}^2 + f_{Y}h_{XX}$$

(now omitting inner variables since always implicitely known) 

https://math.stackexchange.com/questions/2037753/implicit-function-theorem-second-derivative-calculation-help

By assumption $f_{Y}$ is always invertible, so we can solve for the values of $h_{X}$ and $h_{XX}$.

Note that:

$$ h(0) = 0$$ 

since it can't be anything else, otherwise $(0,0)$, wouldn't be in the zero set - a contradiction

### Iterating the derivative

First we consider how to generate equation of the form above, we can split each term at each plus sign into a generalised term of the form:

$$ c \; f_{\alpha X \beta Y} \prod_{i = 1}^{n} {h_{\gamma _{i} X}^{k_{i}} } $$

I.e. a constant multiplied by a mixed partial of $f$ multiplied by a product of varying powers of varying $X$ partials of h

If we can form an encoding scheme for terms like this, and a way to differentiate them with respect to $X$ then we have a method to generate arbitrarlily high order equations to solve.

#### Differentiating

We consider:

$$ \partial_{X} \left[ c \; f_{\alpha X \beta Y} \prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } \right] $$


$$ = c \left[ \left( \partial_{X} f_{\alpha X \beta Y} \right)\prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } + f_{\alpha X \beta Y}\left( \partial_{X} \prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } \right) \right] $$

Splitting this into the two expressions either side of the $+$

$$ \left( \partial_{X} f_{\alpha X \beta Y} \right)\prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } = \left(f_{(\alpha +1)X \beta Y} + f_{\alpha X (\beta + 1)Y} h_{X} \right) \prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}}} $$

And:

$$ f_{\alpha X \beta Y}\left( \partial_{X} \prod_{i = 1}^{n} {h_{\gamma_{i} X}^{k_{i}} } \right) = f_{\alpha X \beta Y} \sum_{i = 1}^{n} {\left( \partial_{X} h_{\gamma_{i} X}^{k_{i}} \right) \prod_{\substack{j = 1 \\ j \neq i}}^{k} h_{\gamma _{j}x}^{k_{j}} } = f_{\alpha X \beta Y} \sum_{i = 1}^{n} {\left( k_{i} \; h_{\gamma _{i} X}^{k_{i}-1} \; h_{(\gamma _{i} - 1)X} \right) \prod_{\substack{j = 1 \\ j \neq i}}^{k} h_{\gamma _{j}x}^{k_{j}} } $$

While very complicated, we see that all the terms we computed stay as sums of expressions of the form:

$$ c \; f_{\alpha X \beta Y} \prod_{i = 1}^{n} {h_{\gamma _{i} X}^{k_{i}} } $$

So inductively we see how the code can proceed

#### Coding the expression block storage + representation

$$ c \; f_{\alpha X \beta Y} \prod_{i = 1}^{n} {h_{\gamma _{i} X}^{k_{i}} } $$

In [76]:
class ExpressionBlock:
    def __init__(self, c = 1, alpha = 0, beta = 0, ktuples = (), func1 = 'f', func2 = 'h', X = 'X', Y = 'Y'):
        self.c = c
        self.alpha = alpha
        self.beta = beta
        self.ktuples = ktuples
        self.func1 = func1
        self.func2 = func2
        self.X = X
        self.Y = Y
        
    def __str__(self):
        # overide representation so can be printed as latex
        if self.c == 0:
            return ""
        
        out = ""
        # add the constant
        if self.c != 1:
            out += str(self.c) + " "
            
        # add f
        out += self.func1
        
        # add partials if there are any
        if (self.alpha != 0) or (self.beta != 0):
            out += "_{"
            # just concatenate them
            # TODO use numbers for values greater than 5
            if self.alpha != 0:
                out += self.X*self.alpha + " "
            if self.beta != 0:
                out += self.Y*self.beta + " "
            out += "} "
            
        for pair in self.ktuples:
            # pairs are (gamma, k)
            if pair[1] == 0:
                # zeroth power of anything is 1, factor of 1 changes nothing - skip
                continue
            out += self.func2
            if pair[0] != 0:
                # gamma
                out += "_{" + self.X*pair[0] + "}"
            if pair[1] != 1:
                # k
                # already ruled out possibility of zero
                out += "^{" + str(pair[1]) + "} " 
                
        # and we are done, here just list out the h terms               
        return out

    def _latex_(self):
        # so works with lprint
        return str(self)

        
        
        
k = ExpressionBlock(c = 4,alpha = 2, beta = 1, ktuples = ((3,4),(0,1),(10,0)))
lprint(k)
k = ExpressionBlock()
lprint(k)

$ 4 f_{XX Y } h_{XXX}^{4} h $

$ f $